In [ ]:
"""
损失函数实验 - 神经网络学习路径第7步

本 Notebook 引入损失函数概念，为神经网络的训练优化提供目标函数。
在前面的 Notebook 中，我们已经构建了完整的前馈神经网络架构：
- 从单个神经元到多层网络
- 抽象出 Layer 和 Network 类
- 引入 Softmax 激活函数用于分类

现在我们需要定义"好坏"的衡量标准 - 损失函数，它将：
1. 衡量模型预测与真实标签之间的差距
2. 为后续的反向传播和参数优化提供梯度信息
3. 指导网络学习正确的分类边界

主要内容：
- 生成二分类数据集（圆形边界分类问题）
- 可视化数据分布
- 实现常用的损失函数（如交叉熵损失）
- 观察损失函数如何量化预测质量
"""


In [45]:
# 生成用于分类的数据，并为每个数据点打标签
import numpy as np
import matplotlib.pyplot as plt
import math
import random

def tag_entry(x,y):
    if x**2+y**2<1:
        tag=0
    else:
        tag=1
    return tag

def create_data(Number_of_Data):
    entry_list=[]
    for i in range (Number_of_Data):
        x=random.uniform(-2,2)
        y=random.uniform(-2,2)
        tag=tag_entry(x,y)
        entry_list.append([x,y,tag])
    return np.array(entry_list)


In [46]:
#可视化
def plot_data(data,title):
    colors=[]
    for i in data[:,2]:
        if i==0:
            colors.append("orange")
        else:
            colors.append("blue")
    plt.scatter(data[:,0],data[:,1],c=colors)
    plt.title(title)
    plt.show()


In [47]:
def activation_ReLU(inputs):
    return np.maximum(0,inputs)

def activation_softmax(inputs):
    max_value=np.max(inputs,axis=1,keepdims=True)
    slided_inputs=inputs-max_value
    exp_values=np.exp(slided_inputs)
    norm_base=np.sum(exp_values,axis=1,keepdims=True)
    norm_values=exp_values/norm_base
    return norm_values

In [48]:
class Layer:

    def __init__(self,n_inputs,n_neurons):
        self.weights=np.random.randn(n_inputs,n_neurons)  
        self.biases=np.random.randn(n_neurons)  

    def forward(self,inputs):
        self.sum=np.dot(inputs,self.weights)+self.biases  
        return self.sum  


In [49]:

class Network:
    def __init__(self,network_shape):
        self.shape=network_shape
        self.layers=[]
        for i in range(len(network_shape)-1):
            layer=Layer(network_shape[i],network_shape[i+1])
            self.layers.append(layer)
    #前馈运算
    def network_forward(self,inputs):
        outputs=[inputs]
        for i in range(len(self.layers)):
            layer_output=self.layers[i].forward(outputs[i])
            if i==len(self.layers)-1:
                layer_output=activation_softmax(layer_output)
            else:
                layer_output=activation_ReLU(layer_output)
            outputs.append(layer_output)
        return outputs

In [50]:
#分类函数
def classify(probabilities):
    classification=np.rint(probabilities[:,1])
    return classification

在这个代码中：
real_matrix[:,1]=real.flatten() - 将真实标签放在第二列（索引1）
real_matrix[:,0]=1-real - 第一列是标签的补集

这意味着：
第一列对应标签0的概率（A类）
第二列对应标签1的概率（B类）

所以如果你的神经网络输出是[P(A类), P(B类)]，那么：
当真实标签是0时，one-hot编码应该是[1, 0]
当真实标签是1时，one-hot编码应该是[0, 1]

关键：你需要确保神经网络的输出顺序与这个one-hot编码的约定一致。如果你的网络第一个输出表示A类，第二个输出表示B类，那么标签0应该对应A类，标签1应该对应B类。
这个约定必须在整个训练过程中保持一致。

In [53]:


def precise_loss_function(predicted,real):
    """
    精确损失函数
    
    参数:
    predicted: 预测值矩阵，形状为(n_samples, 2)
    real: 真实标签，形状为(1, n_samples)或(n_samples,)
    
    返回:
    损失值数组，形状为(n_samples,)
    """

    # 创建真实标签的one-hot编码矩阵
    real_matrix=np.zeros((len(real),2))  # 使用是len(real)
    real_matrix[:,1]=real.flatten()  # 使用flatten()将(1,5)转换为(5,)，赋值给第二列（正类）
    real_matrix[:,0]=1-real  # 第一列为负类（1-正类标签）
    print(real_matrix)
    
    # 计算预测值与真实标签的点积，沿axis=1求和
    product=np.sum(predicted*real_matrix,axis=1)
    
    # 返回损失值：1减去点积
    return 1-product


In [52]:
Number_of_Data=5
data=create_data(Number_of_Data)

inputs=data[:,0:2]
tagets=data[:,2]

net=Network([2,3,4,5,2])
outputs=net.network_forward(inputs)
classification=classify(outputs[-1])

precise_loss_function(outputs[-1],tagets)
print(precise_loss_function(outputs[-1],tagets))

[[0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[0.00151378 0.99931667 0.00122939 0.0023873  0.46932377]
